In [ ]:
import scipy
import numpy as np # manipulate N-dimensional arrays
import pandas as pd # data frame
import matplotlib.pyplot as plt # data plotting
import seaborn # advanced data plotting
from sklearn import preprocessing # basic ML models
# import scipy # scientific computing library

import tensorflow as tf
import keras

import os

In [ ]:
train = pd.read_json ('train.json', lines=True)

In [ ]:
data = train.iloc[np.where(train['SN_filter'] == 1)[0],:]

In [ ]:
data

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
5,5,id_00ab2d761,GGAAAGCGCCGCGGCGGUAGCGGCAGCGAGGAGCGCUACCAAGGCA...,.....(.(((((.(((((((((...........)))))))..(((....,EEEEESISSSSSISSSSSSSSSHHHHHHHHHHHSSSSSSSMMSSSH...,4.136,1,107,68,"[0.1942, 0.2041, 0.1626, 0.1213, 0.10590000000...","[0.2726, 0.2984, 0.21660000000000001, 0.1637, ...","[0.3393, 0.2728, 0.2005, 0.1703, 0.1495, 0.134...","[0.165, 0.20520000000000002, 0.179, 0.1333, 0....","[0.2864, 0.24710000000000001, 0.2222, 0.1903, ...","[0.7642, 1.6641, 1.0622, 0.5008, 0.4107, 0.133...","[0.9559000000000001, 1.9442, 1.0114, 0.5105000...","[1.9554, 2.1298, 1.0403, 0.609, 0.5486, 0.386,...","[0.22460000000000002, 1.7281, 1.381, 0.6623, 0...","[0.5882000000000001, 1.1786, 0.9704, 0.6035, 0..."
6,6,id_00abef1d7,GGAAAACAAUUGCAUCGUUAGUACGACUCCACAGCGUAAGCUGUGG...,.........((((((((......((((((((((((....)))))))...,EEEEEEEEESSSSSSSSIIIIIISSSSSSSSSSSSHHHHSSSSSSS...,2.485,1,107,68,"[0.422, 0.5478000000000001, 0.4749000000000000...","[0.4801, 0.7943, 0.42160000000000003, 0.397300...","[0.9822000000000001, 1.272, 0.6940000000000001...","[0.5827, 0.7555000000000001, 0.5949, 0.4511, 0...","[0.9306000000000001, 1.0496, 0.5844, 0.7796000...","[0.895, 2.3377, 2.2305, 2.003, 1.9006, 1.0373,...","[0.46040000000000003, 3.6695, 0.78550000000000...","[2.7711, 7.365, 1.6924000000000001, 1.43840000...","[1.073, 2.8604000000000003, 1.9936, 1.0273, 1....","[2.0964, 3.3688000000000002, 0.6399, 2.1053, 1..."
7,7,id_00b436dec,GGAAAUCAUCGAGGACGGGUCCGUUCAGCACGCGAAAGCGUCGUGA...,.....(((((((((((..(((((((((..((((....))))..)))...,EEEEESSSSSSSSSSSIISSSSSSSSSIISSSSHHHHSSSSIISSS...,1.727,1,107,68,"[0.4843, 0.5233, 0.4554, 0.43520000000000003, ...","[0.8719, 1.0307, 0.6649, 0.34500000000000003, ...","[0.7045, 0.7775000000000001, 0.5662, 0.4561, 0...","[0.384, 0.723, 0.4766, 0.30260000000000004, 0....","[0.7429, 0.9137000000000001, 0.480400000000000...","[1.1576, 1.5137, 1.3382, 1.5622, 1.2121, 0.295...","[1.6912, 5.2652, 2.3901, 0.45890000000000003, ...","[1.8641, 2.3767, 1.149, 1.0132, 0.9876, 0.0, 0...","[0.49060000000000004, 4.6339, 1.95860000000000...","[1.2852000000000001, 2.5460000000000003, 0.234..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2392,2392,id_feea15e66,GGAAACUCCACAUCUCUACGCCACGAAAGUGGGUAGGAUGGGAGAG...,.....((((.((((.((((.((((....)))))))))))))))).(...,EEEEESSSSBSSSSBSSSSBSSSSHHHHSSSSSSSSSSSSSSSSMS...,3.331,1,

Ajout de donnees

In [ ]:
def prop_base(x):
    v = np.array(list(x))
    prop_A = np.mean(v == 'A')
    prop_C = np.mean(v == 'C')
    prop_U = np.mean(v == 'U')
    return prop_A, prop_C, prop_U

In [ ]:
def prop_loop_type(x):
    v = np.array(list(x))
    prop_S = np.mean(v == 'S')
    prop_E = np.mean(v == 'E')
    prop_H = np.mean(v == 'H')
    prop_B = np.mean(v == 'B')
    prop_I = np.mean(v == 'I')
    prop_X = np.mean(v == 'X')
    return prop_S,prop_E,prop_H,prop_B,prop_I,prop_X

In [ ]:
def get_dist(structure, seq_length):
    dist = np.zeros((seq_length))
    idx = []
    for i, token in enumerate(structure):
        if token == "(":
            idx.append(i)
        elif token == ")":
            j = idx.pop()  # index of the corresponding '(' 
            dist[i] = i-j
            dist[j] = i-j
    return (dist/seq_length)

In [ ]:
def transformation_base_nombre(x):
    if x == 'A':
        return 1
    if x == 'C':
        return 2
    if x == 'G':
        return 3
    if x == 'U':
        return 4

In [ ]:
def transformation_struct_nombre(x):
    if x == '(':
        return 0
    if x == '.':
        return 1
    if x == ')':
        return 2

In [ ]:
def transformation_loop_nombre(x):
    if x == 'S':
        return 0
    if x == 'M':
        return 1
    if x == 'I':
        return 2
    if x == 'B':
        return 3
    if x == 'H':
        return 4
    if x == 'E':
        return 5
    if x == 'X':
        return 6

In [ ]:
train_data = []
for mol_id in data['id'].unique():
    sample_data = data.loc[data['id'] == mol_id]
    pA, pC, pU = prop_base(sample_data['sequence'].values[0])
    pS, pE, pH, pB, pI, pX = prop_loop_type(sample_data['predicted_loop_type'].values[0])
    distance = get_dist(sample_data['structure'].values[0], sample_data['seq_length'].values[0])
    for i in range(sample_data['seq_scored'].values[0]):
        if i == 0:
            sample_tuple = (sample_data['id'].values[0]+'_'+str(i), transformation_base_nombre(sample_data['sequence'].values[0][i]),
                            0, transformation_base_nombre(sample_data['sequence'].values[0][i+1]),
                        pA,pC,pU,
                        transformation_struct_nombre(sample_data['structure'].values[0][i]), 
                        transformation_loop_nombre(sample_data['predicted_loop_type'].values[0][i]),
                        pS,pE,pH,pB,pI,pX,
                        distance[i],
                        i+1,    
                        sample_data['reactivity'].values[0][i], 
                        sample_data['deg_Mg_pH10'].values[0][i],
                        sample_data['deg_pH10'].values[0][i],
                        sample_data['deg_Mg_50C'].values[0][i],
                        sample_data['deg_50C'].values[0][i])
        else:
            sample_tuple = (sample_data['id'].values[0]+'_'+str(i), transformation_base_nombre(sample_data['sequence'].values[0][i]),
                            transformation_base_nombre(sample_data['sequence'].values[0][i-1]), transformation_base_nombre(sample_data['sequence'].values[0][i+1]),
                        pA,pC,pU,
                        transformation_struct_nombre(sample_data['structure'].values[0][i]), 
                        transformation_loop_nombre(sample_data['predicted_loop_type'].values[0][i]),
                        pS,pE,pH,pB,pI,pX,
                        distance[i],
                            i+1,
                        sample_data['reactivity'].values[0][i], 
                        sample_data['deg_Mg_pH10'].values[0][i],
                        sample_data['deg_pH10'].values[0][i],
                        sample_data['deg_Mg_50C'].values[0][i],
                        sample_data['deg_50C'].values[0][i])
        train_data.append(sample_tuple)

In [ ]:
train_data = pd.DataFrame(train_data, columns=['id', 'base','base_prec','base_suiv','prop_base_A','prop_base_C','prop_base_U', 'structure', 'predicted_loop_type',
                                               'prop_loop_S','prop_loop_E','prop_loop_H','prop_loop_B','prop_loop_I','prop_loop_X',
                                               'distance_parite','position','reactivity', 'deg_Mg_pH10',
                                  'deg_pH10', 'deg_Mg_50C', 'deg_50C'])

In [ ]:
df = train_data

In [ ]:
df

,id,base,base_prec,base_suiv,prop_base_A,prop_base_C,prop_base_U,structure,predicted_loop_type,prop_loop_S,prop_loop_E,prop_loop_H,prop_loop_B,prop_loop_I,prop_loop_X,distance_parite,position,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_001f94081_0,3,0,3,0.420561,0.214953,0.186916,1,5,0.429907,0.242991,0.140187,0.009346,0.130841,0.046729,0.000000,1,0.3297,0.7556,2.3375,0.3581,0.6382
1,id_001f94081_1,3,3,1,0.420561,0.214953,0.186916,1,5,0.429907,0.242991,0.140187,0.009346,0.130841,0.046729,0.000000,2,1.5693,2.9830,3.5060,2.9683,3.4773
2,id_001f94081_2,1,3,1,0.420561,0.214953,0.186916,1,5,0.429907,0.242991,0.140187,0.009346,0.130841,0.046729,0.000000,3,1.1227,0.2526,0.3008,0.2589,0.9988
3,id_001f94081_3,1,1,1,0.420561,0.214953,0.186916,1,5,0.429907,0.242991,0.140187,0.009346,0.130841,0.046729,0.000000,4,0.8686,1.3789,1.0108,1.4552,1.3228
4,id_001f94081_4,1,1,1,0.420561,0.214953,0.186916,1,5,0.429907,0.242991,0.140187,0.009346,0.130841,0.046729,0.000000,5,0.7217,0.6376,0.2635,0.7244,0.7877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108047,id_fff546103_63,2,1,4,0.317757,0.271028,0.130841,2,0,0.616822,0.242991,0.093458,0.028037,0.000000,0.000000,0.504673,64,0.0272,0.5059,0.4998,0.2870,0.1820
108048,id_fff546103_64,4,2,1,0.317757,0.271028,0.130841,2,0,0.616822,0.242991,0.093458,0.028037,0.000000,0.000000,0.523364,65,0.0382,0.2025,0.3057,0.2022,0.2038
108049,id_fff546103_65,1,4,3,0.317757,0.271028,0.130841,2,0,0.616822,0.242991,0.093458,0.028037,0.000000,0.000000,0.542056,66,0.0381,0.1827,0.1642,0.2047,0.1120
108050,id_fff546103_66,3,1,2,0.317757,0.271028,0.130841,2,0,0.616822,0.242991,0.093458,0.028037,0.000000,0.000000,0.560748,67,-0.0066,0.3261,0.1654,0.2447,0.1186


In [ ]:
X = df.loc[:,['base','base_prec','base_suiv','prop_base_A','prop_base_C','prop_base_U',
              'structure','predicted_loop_type',
            'prop_loop_S','prop_loop_E','prop_loop_H','prop_loop_B','prop_loop_I','prop_loop_X',
            'distance_parite','position']]

In [ ]:
Y = df.loc[:,["reactivity","deg_Mg_pH10","deg_pH10","deg_Mg_50C","deg_50C"]]

In [ ]:
np.shape(X)

(108052, 16)

In [ ]:
Y

,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0.3297,0.7556,2.3375,0.3581,0.6382
1,1.5693,2.9830,3.5060,2.9683,3.4773
2,1.1227,0.2526,0.3008,0.2589,0.9988
3,0.8686,1.3789,1.0108,1.4552,1.3228
4,0.7217,0.6376,0.2635,0.7244,0.7877
...,...,...,...,...,...
108047,0.0272,0.5059,0.4998,0.2870,0.1820
108048,0.0382,0.2025,0.3057,0.2022,0.2038
108049,0.0381,0.1827,0.1642,0.2047,0.1120
108050,-0.0066,0.3261,0.1654,0.2447,0.1186


In [ ]:
from keras.models import Model,Sequential
from keras.layers import Input, Dense, BatchNormalization, Dropout, ReLU, Activation, Flatten, Conv2D, MaxPooling2D
from keras.backend import clear_session

In [ ]:
clear_session()

In [ ]:
X.shape[1]

16

In [ ]:
model_MLP = Sequential()

model_MLP.add(Dense(256,input_shape = [X.shape[1]])) 
model_MLP.add(ReLU())
model_MLP.add(Dropout(0.2))
model_MLP.add(Dense(256))
model_MLP.add(ReLU())
model_MLP.add(Dropout(0.2))

model_MLP.add(Dense(Y.shape[1]))

In [ ]:
model_MLP.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               4352      
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1

In [ ]:
from keras.optimizers import Adam

In [ ]:
opt = Adam(lr=0.001)

In [ ]:
model_MLP.compile(loss='mean_squared_error',
              optimizer=opt)

In [ ]:
batch_size = 128
epochs = 30
hist_MLP = model_MLP.fit(X, Y,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2)

Epoch 1/30
676/676 [==============================] - 2s 2ms/step - loss: 0.5410 - val_loss: 0.1502
Epoch 2/30
676/676 [==============================] - 1s 2ms/step - loss: 0.1505 - val_loss: 0.1384
Epoch 3/30
676/676 [==============================] - 1s 2ms/step - loss: 0.1419 - val_loss: 0.1306
Epoch 4/30
676/676 [==============================] - 1s 2ms/step - loss: 0.1341 - val_loss: 0.1266
Epoch 5/30
676/676 [==============================] - 1s 2ms/step - loss: 0.1333 - val_loss: 0.1227
Epoch 6/30
676/676 [==============================] - 1s 2ms/step - loss: 0.1261 - val_loss: 0.1197
Epoch 7/30
676/676 [==============================] - 1s 2ms/step - loss: 0.1243 - val_loss: 0.1188
Epoch 8/30
676/676 [==============================] - 1s 2ms/step - loss: 0.1204 - val_loss: 0.1186
Epoch 9/30
676/676 [==============================] - 1s 2ms/step - loss: 0.1217 - val_loss: 0.1189
Epoch 10/30
676/676 [==============================] - 1s 2ms/step - loss: 0.1192 - val_loss: 0.1148

In [ ]:
test = pd.read_json ('test.json', lines=True)

In [ ]:
test_data = []
for mol_id in test['id'].unique():
    sample_data = test.loc[test['id'] == mol_id]
    pA, pC, pU = prop_base(sample_data['sequence'].values[0])
    pS, pE, pH, pB, pI, pX = prop_loop_type(sample_data['predicted_loop_type'].values[0])
    distance = get_dist(sample_data['structure'].values[0], sample_data['seq_length'].values[0])
    for i in range(sample_data['seq_length'].values[0]):
        if i == 0:
            sample_tuple = (sample_data['id'].values[0]+'_'+str(i), transformation_base_nombre(sample_data['sequence'].values[0][i]),
                            0, transformation_base_nombre(sample_data['sequence'].values[0][i+1]),
                        pA,pC,pU,
                        transformation_struct_nombre(sample_data['structure'].values[0][i]), 
                        transformation_loop_nombre(sample_data['predicted_loop_type'].values[0][i]),
                        pS,pE,pH,pB,pI,pX,
                        distance[i],
                        i+1)
        elif i == sample_data['seq_length'].values[0] - 1:
            sample_tuple = (sample_data['id'].values[0]+'_'+str(i), transformation_base_nombre(sample_data['sequence'].values[0][i]),
                            transformation_base_nombre(sample_data['sequence'].values[0][i-1]), 0,
                        pA,pC,pU,
                        transformation_struct_nombre(sample_data['structure'].values[0][i]), 
                        transformation_loop_nombre(sample_data['predicted_loop_type'].values[0][i]),
                        pS,pE,pH,pB,pI,pX,
                        distance[i],
                        i+1)
        else:
            sample_tuple = (sample_data['id'].values[0]+'_'+str(i), transformation_base_nombre(sample_data['sequence'].values[0][i]),
                            transformation_base_nombre(sample_data['sequence'].values[0][i-1]), transformation_base_nombre(sample_data['sequence'].values[0][i+1]),
                        pA,pC,pU,
                        transformation_struct_nombre(sample_data['structure'].values[0][i]), 
                        transformation_loop_nombre(sample_data['predicted_loop_type'].values[0][i]),
                        pS,pE,pH,pB,pI,pX,
                        distance[i],
                            i+1)
        test_data.append(sample_tuple)

In [ ]:
test_data = pd.DataFrame(test_data, columns=['id', 'base','base_prec','base_suiv','prop_base_A','prop_base_C','prop_base_U', 'structure', 'predicted_loop_type',
                                               'prop_loop_S','prop_loop_E','prop_loop_H','prop_loop_B','prop_loop_I','prop_loop_X',
                                               'distance_parite','position'])

In [ ]:
new_test = test_data.loc[:,['base','base_prec','base_suiv','prop_base_A','prop_base_C','prop_base_U',
              'structure','predicted_loop_type',
            'prop_loop_S','prop_loop_E','prop_loop_H','prop_loop_B','prop_loop_I','prop_loop_X',
            'distance_parite','position']]

In [ ]:
type(new_test)

pandas.core.frame.DataFrame

In [ ]:
y_pred = model_MLP.predict(new_test)

In [ ]:
tab = pd.read_csv("sample_submission.csv")

In [ ]:
y_pred_id = test_data["id"].values

In [ ]:
df_id = pd.Series(y_pred_id)

In [ ]:
df_DNN = pd.DataFrame(y_pred, columns=list(tab.columns)[1:])

In [ ]:
df_DNN

,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0.695006,0.739120,2.179359,0.535164,0.744113
1,1.992028,2.801833,3.654327,2.629273,2.252679
2,1.357069,0.941986,1.162849,1.227386,1.233486
3,1.050834,0.773974,0.926788,1.026403,1.036275
4,1.009936,0.756001,0.882532,0.990773,0.998793
...,...,...,...,...,...
457948,0.666842,1.149931,1.134735,1.093953,0.813368
457949,0.850094,1.135665,0.863150,1.290382,0.856463
457950,0.922361,0.942450,0.761848,1.122113,0.804904
457951,0.689272,1.210707,1.188426,1.153729,0.846224


In [ ]:
df_test = pd.concat((df_id, df_DNN), axis = 1)

In [ ]:
df_test.rename(columns={0: list(tab.columns)[0]}, inplace=True)

In [ ]:
df_test.to_csv('DNN_avec_ajout_sans_filtrage.csv', index=False)

In [ ]:
from google.colab import files

files.download('DNN_avec_ajout_sans_filtrage.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_filtre = df_test

In [ ]:
df_filtre.iloc[np.where(df_test['reactivity'] < 0)[0],1] = 0
df_filtre.iloc[np.where(df_test['deg_Mg_pH10'] < 0)[0],2] = 0
df_filtre.iloc[np.where(df_test['deg_pH10'] < 0)[0],3] = 0
df_filtre.iloc[np.where(df_test['deg_Mg_50C'] < 0)[0],4] = 0
df_filtre.iloc[np.where(df_test['deg_50C'] < 0)[0],5] = 0

In [ ]:
df_filtre.to_csv('DNN_avec_ajout_avec_filtrage.csv', index=False) 

In [ ]:
files.download('DNN_avec_ajout_avec_filtrage.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>